In [41]:
pip install PyMuPDF opencv-python numpy


In [42]:
import pdfplumber

In [43]:
pip install pdfplumber

In [44]:
pip install pdf2image pytesseract

In [45]:
import json
from pdf2image import convert_from_path
from PIL import Image

In [46]:
def extract_text_from_bbox(image, bbox):
    x1, y1, x2, y2, x3, y3, x4, y4 = bbox

    # Ensure bounding box coordinates are within image dimensions
    img_width, img_height = image.size
    x1, x2, x3, x4 = map(lambda x: max(0, min(x, img_width)), [x1, x2, x3, x4])
    y1, y2, y3, y4 = map(lambda y: max(0, min(y, img_height)), [y1, y2, y3, y4])

    # Extract text within the corrected bounding box
    roi = image.crop((x1, y1, x3, y3))
    return pytesseract.image_to_string(roi).strip()

def extract_information_from_images(pdf_path, bounding_boxes):
    # Convert PDF to images
    images = convert_from_path(pdf_path)

    extracted_info = {}

    for page_number, image in enumerate(images, start=1):
        for box_info in bounding_boxes:
            for label, bbox in box_info.items():
                text_in_bbox = extract_text_from_bbox(image, bbox)
                key = f"{label}_Page_{page_number}"
                extracted_info[key] = text_in_bbox

    return extracted_info

In [47]:
# Install Poppler
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [48]:
# Install pytesseract and Tesseract OCR engine
!apt install tesseract-ocr
!pip install pytesseract

# Import pytesseract after installation
import pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [51]:
import fitz  # PyMuPDF
from PIL import Image
import pytesseract


In [52]:
json_data = [
    {"06/10/2021": [6.8505, 0.2667, 7.6672, 0.2634, 7.6638, 0.4267, 6.8471, 0.4267]},
    {"Company": [1.3234, 0.6234, 1.9968, 0.6267, 1.9801, 0.8167, 1.3201, 0.8034]},
    # Add other bounding boxes as needed
]


In [53]:
pdf_path = "sample_invoice.pdf"  # Replace with the actual path to your PDF file
pdf_document = fitz.open(pdf_path)

In [54]:
def extract_text_from_bbox(page, bbox):
    x1, y1, x2, y2, x3, y3, x4, y4 = bbox

    # Extract text within the bounding box
    rect = fitz.Rect(x1, y1, x3, y3)
    roi = page.get_text("text", clip=rect)
    return roi.strip()

In [55]:
page_number = 0  # Single-page PDF, so only one page (0-based index)
page = pdf_document[page_number]
image = page.get_pixmap()

# Process each bounding box on the current page
for bbox_info in json_data:
    key, bbox = next(iter(bbox_info.items()))
    text = extract_text_from_bbox(page, bbox)
    print(f"{key}: {text}")

06/10/2021: 
Company: 


In [56]:
pil_image = Image.frombytes("RGB", (image.width, image.height), image.samples)


In [58]:
ocr_text = pytesseract.image_to_string(pil_image).strip()
print(f"OCR Text (Page {page_number + 1}): {ocr_text}")

# Close the PDF document
#pdf_document.close()

OCR Text (Page 1): 06/10/2021

 

7 INVO-005
Gey Sample Invoice
Billing Information Shipping Information
Company Name Name
‘ABC Company John Smith ‘Sam K. Smith
‘Address Address
111 Pine Street, Suite 1815 111 Pine Street, Suite 1815
San Francisco, CA, 94111 San Francisco, CA, 94111
Phone Number
(123) 123-1232
Email

John@example.com

 

Description Quantity Unt Price Total
ProductService 1 Sink 2 100 200
ProductService2 Nest Smart Fier 1 150 150
ProductiService3 Labor Fee 1 50 $50
ProducvSenices Service Fee 1 8 925

Total: $425
